In [5]:
import pandas as pd
import numpy as np
import requests
from datetime import date, timedelta
import os
import re
import tarfile
import zipfile
import bz2
import glob
import logging
from unittest.mock import patch
from typing import List, Set, Dict, Tuple, Optional
from itertools import zip_longest
import plotly.express as px

import betfairlightweight
from betfairlightweight import StreamListener
from betfairlightweight.resources.bettingresources import (
    PriceSize,
    MarketBook
)

%config IPCompleter.greedy=True

In [6]:
# Current wd
# print("Current Working Directory " , os.getcwd())

In [7]:
try:
    # Change the current working Directory    
    os.chdir("C:\\")
    print("Directory changed")
    print("Current Working Directory " , os.getcwd())
except OSError:
    print("Can't change the Current Working Directory")   

Directory changed
Current Working Directory  C:\


In [15]:
trading = betfairlightweight.APIClient("username", "password")

# create listener
listener = StreamListener(max_latency=None)

In [16]:
def ladder_traded_volume(ladder):
                return(sum([rung.size for rung in ladder]))

# returns size traded for all sizes for all runners

In [17]:
def top_3_ladder(availableLadder):
#                 out = {}
#                 out = []
#                 price = []
#                 volume = []
    if len(availableLadder) == 0:
        out = []
        return(out)        
    else:
        for rung in availableLadder[0:1]:
#                         price.append(rung.price)
#                         volume.append(rung.size)
#                         out["price"] = price
#                         out["volume"] = volume                        

            # New experimental code
            out = [price,size]
                        

# Applies the function that finds the top 3 prices / stakes for the t-3 mins market book
# Sometimes t-3 mins market book is empty
try:
    atb_ladder_3m = [ top_3_ladder(runner.ex.available_to_back) for runner in t3m_market.runners]
    atl_ladder_3m = [ top_3_ladder(runner.ex.available_to_lay) for runner in t3m_market.runners]
except:
    atb_ladder_3m = {}
    atl_ladder_3m = {}

In [18]:
def load_markets(file_paths: List[str]):
    # loops over each of the strings in the list
    for file_path in file_paths:
        # Prints each of the stirngs in the list
        print(file_path)
        # If the string exists then
        if os.path.isdir(file_path):
            # https://docs.python.org/3/library/glob.html
            # Return a of path names that match pathname
            # glob.iglob(pathname, *, recursive=False)
            # If recursive is true, the pattern “**” will match any files and zero or more directories, subdirectories and symbolic links to directories.
            # basically it finds all files in the path that ends with .bz2 within the file path and returns it
            for path in glob.iglob(file_path + '**/**/*.bz2', recursive=True):
                f = bz2.BZ2File(path, 'rb')
                yield f
                f.close()
        # if the string path doesn't exist, check if it is a file
        elif os.path.isfile(file_path):
            # if it is a file then takes its path go up one step and then basically iterate throught everything in that upper file
            ext = os.path.splitext(file_path)[1]
            # iterate through a tar archive
            if ext == '.tar':
                with tarfile.TarFile(file_path) as archive:
                    for file in archive:
                        yield bz2.open(archive.extractfile(file))
            # or a zip archive
            elif ext == '.zip':
                with zipfile.ZipFile(file_path) as archive:
                    for file in archive.namelist():
                        yield bz2.open(archive.open(file))

    return None

In [19]:
%%time
market_path = ['testfiles']
def test(market_path):
    for file_obj in load_markets(market_path):
        # Create trading.streaming but this time we are using .create_historical_generator_stream
        # Assuming this is the version for treading.streaming but for historical data
        # Instantiate a "stream" object
        stream = trading.streaming.create_historical_generator_stream(
            file_path=file_obj,
            listener=listener)
        print(stream)

        with patch("builtins.open", lambda f, _: f):   

            # Create generator
            gen = stream.get_generator()

            # For each group of market books in the generator
            for market_books in gen():
                # For each individual market book in the group of market books
                for market_book in market_books:
                    seconds_to_start = (market_book.market_definition.market_time - market_book.publish_time).total_seconds()
                    if (seconds_to_start < 0 or seconds_to_start > 300):
                        continue
                    market_id = market_book.market_id
                    for runner in market_book.runners:
                        lst = []
                        selection_id = runner.selection_id
                        selection_name = next((rd.name for rd in market_book.market_definition.runners if rd.selection_id == runner.selection_id), None)
                        selection_status = runner.status
                #         sp = runner.sp.actual_sp This seems useless

                        selection_traded_volume = ladder_traded_volume(runner.ex.traded_volume)
                        try:
#                             atb_ladder_3m = [top_3_ladder(runner.ex.available_to_back)]
#                             atl_ladder_3m = [top_3_ladder(runner.ex.available_to_lay)]
                            best_back_price = runner.ex.available_to_back[0].price
                            best_back_size = runner.ex.available_to_back[0].size
                            best_lay_price = runner.ex.available_to_lay[0].price
                            best_lay_size = runner.ex.available_to_lay[0].size
#                             best_back = [runner.ex.available_to_back[0].price,runner.ex.available_to_back[0].volume]
#                             best_lay = [runner.ex.available_to_lay[0]]
                #             selection_traded_volume = runner.ex.traded_volume
                        except:
                            best_back_price = []
                            best_back_volume = []
                            best_lay_price = []
                            best_lay_volume = []

                #         lastPriceTraded: The price of the most recent bet matched on this selection    
                        try:
                            last_price_traded = runner.last_price_traded
                        except:
                            last_price_traded = None

                        yield [market_id,seconds_to_start,selection_id,selection_name,selection_status,best_back_price, best_back_size,best_lay_price,best_lay_size,selection_traded_volume,last_price_traded]

Wall time: 0 ns


In [20]:
%%time
df = pd.DataFrame(list(test(market_path)), columns = ['market_id','seconds_to_start','selection_id','selection_name','selection_status','best_back_price', 'best_back_volume','best_lay_price','best_lay_size','selection_traded_volume','last_price_traded'])
df.to_csv('2021_07_02.csv')

testfiles
Wall time: 1.88 s


In [21]:
df

,market_id,seconds_to_start,selection_id,selection_name,selection_status,best_back_price,best_back_volume,best_lay_price,best_lay_size,selection_traded_volume,last_price_traded
0,1.184880762,299.846,37455069,1. Alvin The Bold,ACTIVE,1.23,110.82,1.25,1.62,943.36,1.24
1,1.184880762,299.846,39905643,2. Bye Bye Charlie,ACTIVE,6.60,5.10,8.40,19.69,38.13,7.00
2,1.184880762,299.846,39657966,3. Angela Sistina,ACTIVE,2.18,5.00,2.28,13.54,161.18,2.16
3,1.184880762,299.846,39955056,4. Carlisle Bay,ACTIVE,3.60,3.00,3.95,1.62,10.83,3.60
4,1.184880762,299.846,40031905,5. Contamination,ACTIVE,1.99,7.00,2.10,10.83,134.16,1.98
...,...,...,...,...,...,...,...,...,...,...,...
15155,1.184880762,0.043,39955056,4. Carlisle Bay,ACTIVE,3.15,4.49,3.25,0.13,352.28,2.96
15156,1.184880762,0.043,40031905,5. Contamination,ACTIVE,2.18,10.00,2.20,27.09,267.62,2.16
15157,1.184880762,0.043,40031906,6. Tip Top Timing,ACTIVE,3.75,23.52,4.30,18.76,376.91,3.75
15158,1.184880762,0.043,35372517,7. Galierro,ACTIVE,1.72,12.04,1.77,2.36,592.36,1.72
